### Scrape Shallalist Data

In [ ]:
import os
import logging
import requests
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import mapply

mapply.init(
    n_workers=4,
    chunk_size=100,
    max_chunks_per_worker=0,
    progressbar=True
)


logging.basicConfig(filename='scrape_all.log',
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.ERROR)

In [ ]:
shalla = pd.read_csv("./shallalist_all.csv")
shalla

In [ ]:
# Drop duplicates
shalla.drop_duplicates(subset = ['domain'], inplace = True)
shalla.shape

In [ ]:
shalla['status_code'] = 0
shalla

In [ ]:
def download_file(url, fn):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    # NOTE the stream=True parameter below
    status = -1
    with requests.get(url, timeout=5, verify=False, headers=headers, stream=True) as r:
        #print(r.headers)
        r.raise_for_status()
        with open(fn, 'wb') as f:
            count = 0
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
                count += 1
                if count > 1024:
                    # break if size > 8MB
                    break
        status = r.status_code
    return status

In [ ]:
def scrape_page(r):
    try:
        fn = 'scrape_all/' + r['domain'] + '.html'
        if os.path.exists(fn):
            return 0
        else:
            url = 'http://' + r['domain']
            return download_file(url, fn)
    except Exception as e:
        #print('ERROR:', fn)
        with open(fn, 'w') as f:
            f.write('ERROR: %r' % e)
        logging.error(e)
        return -1

In [ ]:
shalla['status_code'] = shalla.mapply(scrape_page, axis=1)

In [ ]:
shalla

In [ ]:
shalla.groupby(['status_code']).size()